In [4]:
# GPU Sanity Check
import torch, platform
print("OS:", platform.platform())
print("PyTorch:", torch.__version__)
print("torch.version.cuda:", torch.version.cuda)
print("CUDA available?:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
x = torch.randn(1024,1024, device="cuda" if torch.cuda.is_available() else "cpu")
y = x @ x.T
print("Tensor device:", y.device)

try:
    import onnxruntime as ort
    print("onnxruntime:", ort.__version__)
    print("providers:", ort.get_available_providers())
except Exception as e:
    print("onnxruntime not ready:", e)


OS: Windows-10-10.0.26100-SP0
PyTorch: 2.5.1+cu121
torch.version.cuda: 12.1
CUDA available?: True
GPU: NVIDIA GeForce RTX 3050 Ti Laptop GPU
Tensor device: cuda:0
onnxruntime not ready: No module named 'onnxruntime'


In [1]:
import pandas as pd, gradio, fastapi, httpx
print("✅ pandas:", pd.__version__)
print("✅ gradio:", gradio.__version__)
print("✅ fastapi:", fastapi.__version__)


c:\Users\Motasem\Desktop\motasem today\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ pandas: 2.2.2
✅ gradio: 4.44.0
✅ fastapi: 0.115.0


In [3]:
# agent_gate_app.py
# ================== بوابة تحقق + Agent بملف واحد ==================
import os, io, time, hashlib, json, webbrowser
from pathlib import Path
from typing import Optional, Dict, Any, List

import pandas as pd
from PIL import Image

import pyttsx3
import httpx

from fastapi import FastAPI, UploadFile, Form
from fastapi.responses import JSONResponse
import gradio as gr

# ---------- مسارات أساسية ----------
ROOT = Path(".").resolve()
LOGS = ROOT / "logs"; LOGS.mkdir(parents=True, exist_ok=True)
AUDIT_CSV = LOGS / "audit_log.csv"

# ---------- TTS (صوت للوكيل بعد الدخول) ----------
_tts = pyttsx3.init()
def say(text: str):
    try:
        _tts.say(text)
        _tts.runAndWait()
    except Exception:
        pass

# ---------- دالة التحقق (بدّلها بالموديل الحقيقي لاحقاً) ----------
ALLOWLIST = {"stark", "admin", "demo"}  # أمثلة — امسحها لاحقاً

def fake_similarity(img: Image.Image) -> float:
    # نحسب هاش للصورة لننتج "درجة" ثابتة الشكل (مجرد Placeholder)
    img = img.convert("RGB").resize((64, 64))
    b = io.BytesIO()
    img.save(b, format="JPEG", quality=85)
    h = hashlib.md5(b.getvalue()).hexdigest()
    # حوّل جزء من الهاش لدرجة بين [0,1]
    v = int(h[:6], 16) / 0xFFFFFF
    return float(v)

def verify_face(username: str, image_bytes: bytes) -> Dict[str, Any]:
    """
    ⬅️ بدّل محتوى هذه الدالة لاحقاً لاستدعاء الموديل الحقيقي:
       - مثال REST: أرسل image_bytes + username إلى http://localhost:8001/verify
       - مثال ONNX/PyTorch: استخرج embedding وقارن مع قالب المستخدم
    """
    try:
        img = Image.open(io.BytesIO(image_bytes))
    except Exception:
        return {"ok": False, "reason": "invalid_image", "score": 0.0}

    score = fake_similarity(img)
    # قاعدة مؤقتة: لازم الاسم ضمن allowlist ودرجة >= 0.55
    granted = (username.lower() in ALLOWLIST) and (score >= 0.55)
    return {"ok": granted, "reason": "ok" if granted else "mismatch", "score": round(score, 3)}

# ---------- Audit Logging ----------
def append_audit(username: str, decision: str, score: float, reason: str, meta: Dict[str, Any]):
    row = {
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
        "username": username,
        "decision": decision,
        "score": score,
        "reason": reason,
        "model_version": meta.get("model_version", "stub-0.1"),
        "client": meta.get("client", "web"),
    }
    if AUDIT_CSV.exists():
        df = pd.read_csv(AUDIT_CSV)
        df.loc[len(df)] = row
    else:
        df = pd.DataFrame([row])
    df.to_csv(AUDIT_CSV, index=False)

# ---------- FastAPI backend ----------
app = FastAPI(title="FaceGate API")

@app.post("/verify")
async def api_verify(username: str = Form(...), image: UploadFile = Form(...)):
    if not username or image is None:
        return JSONResponse({"ok": False, "error": "missing_fields"}, status_code=400)
    img_bytes = await image.read()
    if len(img_bytes) < 1024:
        append_audit(username, "denied", 0.0, "no_image", {"client": "api"})
        return {"ok": False, "decision": "denied", "score": 0.0, "msg": "No/invalid image"}
    res = verify_face(username, img_bytes)
    decision = "granted" if res["ok"] else "denied"
    append_audit(username, decision, res["score"], res["reason"], {"client": "api"})
    return {"ok": True, "decision": decision, "score": res["score"], "reason": res["reason"]}

@app.get("/audit")
async def api_audit():
    if not AUDIT_CSV.exists():
        return {"records": []}
    return {"records": pd.read_csv(AUDIT_CSV).tail(200).to_dict(orient="records")}

# ---------- أدوات الوكيل (Agent Tools) ----------
def tool_webhook(url: str, message: str) -> str:
    try:
        r = httpx.post(url, json={"message": message}, timeout=5.0)
        return f"Webhook status: {r.status_code}"
    except Exception as e:
        return f"Webhook failed: {e}"

def tool_create_task(title: str, due: str) -> str:
    TASKS = LOGS / "tasks.json"
    lst = json.loads(TASKS.read_text("utf-8")) if TASKS.exists() else []
    lst.append({"title": title, "due": due, "ts": time.time()})
    TASKS.write_text(json.dumps(lst, ensure_ascii=False, indent=2), encoding="utf-8")
    return f"✅ Task saved: {title} (due {due})"

def tool_summarize(text: str) -> str:
    # تبسيط سريع (مش LLM) — Placeholder
    s = " ".join(text.strip().split())
    words = s.split()
    if len(words) <= 40: return s
    return " ".join(words[:40]) + " …"

def tool_open(url: str) -> str:
    try:
        webbrowser.open(url)
        return f"Opened: {url}"
    except Exception as e:
        return f"Open failed: {e}"

# ---------- Gradio UI: Login → Agent ----------
with gr.Blocks(theme=gr.themes.Soft(primary_hue="indigo")) as gr_app:
    gr.Markdown("# 🔐 FaceGate → 🧠 J.A.R.V.I.S (Demo)\n> Upload + Verify → Enter Agent")
    with gr.Tab("Login / Verify"):
        username = gr.Textbox(label="Username", value="")
        img = gr.Image(label="Face Image", type="filepath")
        out_decision = gr.HTML()
        out_score = gr.Number(label="Score", interactive=False)
        btn = gr.Button("Verify")

        def do_verify(u, path):
            if not u or not path:
                return gr.update(value="<span style='color:red'>Missing fields</span>"), 0.0
            with open(path, "rb") as f:
                files = {"image": ("upload.jpg", f, "image/jpeg")}
                data = {"username": u}
                try:
                    r = httpx.post("http://127.0.0.1:8000/verify", data=data, files=files, timeout=10.0)
                    rj = r.json()
                except Exception as e:
                    return gr.update(value=f"<span style='color:red'>API error: {e}</span>"), 0.0

            if not rj.get("ok", False):
                return gr.update(value=f"<span style='color:red'>Denied: {rj.get('msg','error')}</span>"), float(rj.get("score", 0.0))
            decision = rj["decision"]; score = float(rj["score"])
            color = "green" if decision=="granted" else "red"
            if decision == "granted":
                say(f"Welcome {u}. Access granted.")
            else:
                say("Access denied.")
            html = f"<b>Decision: <span style='color:{color}'>{decision.upper()}</span></b>"
            return gr.update(value=html), score

        btn.click(do_verify, [username, img], [out_decision, out_score])

    with gr.Tab("Agent (post-login)"):
        gr.Markdown("### أدوات سريعة")
        with gr.Row():
            url = gr.Textbox(label="Webhook URL", placeholder="https://example.com/hook")
            msg = gr.Textbox(label="Message")
        run_hook = gr.Button("Send Webhook")
        hook_out = gr.Textbox(label="Result", interactive=False)
        run_hook.click(lambda u, m: tool_webhook(u, m), [url, msg], [hook_out])

        with gr.Row():
            t_title = gr.Textbox(label="Task title")
            t_due = gr.Textbox(label="Due (e.g., 2025-09-20)")
        run_task = gr.Button("Create Task")
        task_out = gr.Textbox(label="Result", interactive=False)
        run_task.click(lambda a, b: tool_create_task(a, b), [t_title, t_due], [task_out])

        with gr.Row():
            sum_in = gr.Textbox(label="Summarize text")
        run_sum = gr.Button("Summarize")
        sum_out = gr.Textbox(label="Summary", interactive=False)
        run_sum.click(tool_summarize, [sum_in], [sum_out])

        with gr.Row():
            open_url = gr.Textbox(label="Open URL")
        run_open = gr.Button("Open in Browser")
        open_out = gr.Textbox(label="Result", interactive=False)
        run_open.click(tool_open, [open_url], [open_out])

        gr.Markdown("#### Audit Trail (last 200)")
        audit_btn = gr.Button("Refresh Audit")
        audit_tbl = gr.Dataframe(headers=["timestamp","username","decision","score","reason","model_version","client"], row_count=5)
        def load_audit():
            if not AUDIT_CSV.exists():
                return []
            return pd.read_csv(AUDIT_CSV).tail(200)
        audit_btn.click(load_audit, outputs=[audit_tbl])

# اربط Gradio داخل FastAPI
app = gr.mount_gradio_app(app, gr_app, path="/")


c:\Users\Motasem\Desktop\motasem today\.venv\Lib\site-packages\gradio\analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(
